## Import packages

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import xgboost as xgb

from datetime import date, datetime
import time
import calendar

import matplotlib.pyplot as plt
import seaborn as sns

## Load dataset

In [3]:
path = '../input/store-sales-time-series-forecasting/'

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
sub = pd.read_csv(path + 'sample_submission.csv')

oil = pd.read_csv(path + 'oil.csv')
holiday = pd.read_csv(path + 'holidays_events.csv')
store = pd.read_csv(path + 'stores.csv')
tran = pd.read_csv(path + 'transactions.csv')

## EDA

In [14]:
data_oil = train[['date','sales']].copy()
data_oil = data_oil.merge(oil, on='date', how='left').fillna('0')
data_oil['dcoilwtico'] = data_oil['dcoilwtico'].astype(float)
data_oil.corr()

#### The correlation coefficient  between dcoilwtico and sales suggests that there's almost no relationship between these two variables.

In [22]:
ax = train.hist(column = ['sales'],bins=100, alpha=0.5)

In [23]:
ax = train[train.sales != 0].hist(column = ['sales'],bins=100, alpha=0.5)

#### The distribution of sales is positively skewed.

#### The impact of family feature on sales

In [37]:
plt.figure(figsize=(20,8))
data_familes = train[['family','sales']].groupby('family').agg('mean').reset_index(level=0)
data_familes.sort_values(by = 'sales', inplace = True)
data_familes.head()
ax = sns.barplot(y='family', x='sales', data=data_familes)

#### Check the influence of holiday on sales

In [42]:
data_holiday = train[['date','sales']].copy()
data_holiday = data_holiday.merge(holiday, on='date', how='left')
data_familes = data_holiday.groupby('type')['sales'].agg('mean').reset_index(level=0)
data_familes.sort_values(by = 'sales', inplace = True)
ax = sns.barplot(x='type', y='sales', data=data_familes)

#### The average sales of holidays is smaller than the average sales of work days.

#### Check the influence of transactions on sales

In [59]:
data_tran = train[['date', 'store_nbr','sales']].copy()
data_tran = data_tran.merge(tran, on=['date', 'store_nbr'], how='left').fillna(0.0)
data_tran['sales'] = data_tran['sales'].astype(int)
data_tran['transactions'] = data_tran['transactions'].astype(int)
ax = data_tran.plot.scatter(x='transactions',y='sales',c='DarkBlue',figsize=(10,5))

In [61]:
data_tran = train[['date', 'store_nbr','sales']].copy()
data_tran = data_tran.merge(tran, on=['date', 'store_nbr'], how='left').fillna(0.0)
data_tran['sales'] = np.log(data_tran['sales'] + 1).astype(int)
data_tran['transactions'] = np.log(data_tran['transactions'] + 1).astype(int)
ax = data_tran.plot.scatter(x='transactions',y='sales',c='DarkBlue',figsize=(10,5))

## Preprocess datasets

* add weekday, year, month, day and payday

In [ ]:
def preprocess_train(df):
    df['date'] = df['date'].map(lambda x: date.fromisoformat(x))
    df['weekday'] = df['date'].map(lambda x: x.weekday())
    df['year'] = df['date'].map(lambda x: x.year)
    df['month'] = df['date'].map(lambda x: x.month)
    df['day'] = df['date'].map(lambda x: x.day)
    df['eomd'] = df['date'].map(lambda x: calendar.monthrange(x.year, x.month)[1])
    df['payday'] = ((df['day'] == df['eomd'])|(df['day'] == 15)).astype(int)
    df.drop(['id', 'eomd'], axis=1, inplace=True)
    return df

train = preprocess_train(train)
test = preprocess_train(test)

* fill in NA values with month average oil price

In [ ]:
def preprocess_oil(oil):
    oil['month'] = oil['date'].map(lambda x: int(x.replace('-', '')[:6]))
    oil['month_avg'] = oil.groupby('month')['dcoilwtico'].transform('mean')
    oil['tmp'] = oil['dcoilwtico'].map(np.isnan)
    oil['month_avg'] = oil['tmp'] * oil['month_avg']
    oil['dcoilwtico'].fillna(0, inplace=True)
    oil['dcoilwtico'] = oil['dcoilwtico'] + oil['month_avg']
    oil = oil.drop(['month', 'month_avg', 'tmp'], axis=1)
    oil['date'] = oil['date'].map(lambda x: date.fromisoformat(x))
    return oil

oil = preprocess_oil(oil)

* separate into three holiday types (national, regional and local) and event (FIFA World Cup etc)

In [ ]:
def preprocess_holiday(df):
    df['date'] = df['date'].map(lambda x: date.fromisoformat(x))
    df = df[(df['transferred']==False)&(df['type']!='Work Day')]
    event = df[df['type']=='Event']
    earthquake = event[event['description'].str.startswith('Terremoto Manabi')]
    event = event[event['description'].str.startswith('Terremoto Manabi')==False]
    return df, event, earthquake

holiday, event, earthquake = preprocess_holiday(holiday)

event = event[['date', 'description']]
event.rename({'description': 'event_name'}, axis=1, inplace=True)

earthquake = earthquake[['date', 'description']]
earthquake.rename({'description': 'earthquake'}, axis=1, inplace=True)

h_local = holiday[holiday['locale']=='Local']
h_local = h_local[['date', 'locale_name', 'description']]
h_local = h_local.rename({'locale_name': 'city', 'description': 'local_holiday_name'}, axis=1)

h_regional = holiday[holiday['locale']=='Regional']
h_regional = h_regional[['date', 'locale_name', 'description']]
h_regional = h_regional.rename({'locale_name': 'state', 'description': 'regional_holiday_name'}, axis=1)

h_national = holiday[holiday['locale']=='National']
h_national = h_national[['date', 'description']]
h_national = h_national.rename({'description': 'national_holiday_name'}, axis=1)

## Merge datasets

In [ ]:
def merge_tables(df):
    df = df.merge(oil, on='date', how='left')
    df = df.merge(store, on='store_nbr', how='left')
    df = df.merge(event, on='date', how='left').fillna('0')
    df = df.merge(earthquake, on='date', how='left').fillna('0')
    df = df.merge(h_local, on=['date', 'city'], how='left').fillna('0')
    df = df.merge(h_regional, on=['date', 'state'], how='left').fillna('0')
    df = df.merge(h_national, on='date', how='left').fillna('0')
    df = df.merge(tran, on=['date', 'store_nbr'], how='left').fillna(0)

    return df

train = merge_tables(train)
test = merge_tables(test)

In [ ]:
train['dcoilwtico'] = train['dcoilwtico'].astype(float)
test['dcoilwtico'] = test['dcoilwtico'].astype(float)

## Label encoding

In [ ]:
cat_features = ['family', 'store_nbr', 'city', 'state', 'type', 'cluster',
                'event_name', 'earthquake', 'local_holiday_name', 'regional_holiday_name', 'national_holiday_name']
for col in cat_features:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

## Set train period and validation period

In [ ]:
def preprocess_dataset(df, train_date: list, valid_date: list):
    df['is_train'] = df['date'].map(lambda x: x in train_date)
    df['is_valid'] = df['date'].map(lambda x: x in valid_date)
    return df


train_date = train['date'].unique()[-227:-31].tolist()
valid_date = train['date'].unique()[-31:].tolist()
train = preprocess_dataset(train, train_date, valid_date)

In [ ]:
print('train date from {} to {}'.format(min(train_date), max(train_date)))
print('valid date from {} to {}'.format(min(valid_date), max(valid_date)))

## Set X and y

In [ ]:
y = np.log(train['sales'] + 1)
X_train = train.drop(['date', 'sales', 'year'], axis=1)
X_test = test.drop(['date', 'year'], axis=1)

In [ ]:
X_train

## Run XGB

In [ ]:
# extract train and valid dataset
trn_idx = X_train[X_train['is_train']==True].index.tolist()
val_idx = X_train[X_train['is_valid']==True].index.tolist()

X_tr = X_train.loc[trn_idx, :].drop(['is_train', 'is_valid'], axis=1)
X_val = X_train.loc[val_idx, :].drop(['is_train', 'is_valid'], axis=1)
y_tr = y[trn_idx]
y_val = y[val_idx]
    


In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_importance
xgb = XGBRegressor(
    max_depth=6,
    n_estimators=1000,
    min_child_weight=50, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    tree_method='gpu_hist',
    eta=0.2,    
    seed=42)

xgb.fit(
    X_tr, 
    y_tr, 
    eval_metric="rmse", 
    eval_set=[(X_tr, y_tr), (X_val, y_val)], 
    verbose=True,
    early_stopping_rounds=10)

In [ ]:
y_pred = xgb.predict(X_test)

In [ ]:
sub['sales'] = np.exp(np.array(list(map(lambda x: max(x, 0), y_pred))))-1

In [ ]:
xgb_score = round(xgb.score(X_val, y_val) * 100, 2)
xgb_score

## LSTM

In [ ]:
X_tr = X_tr.to_numpy().reshape(X_tr.shape[0], X_tr.shape[1], 1)
X_val = X_val.to_numpy().reshape(X_val.shape[0], X_val.shape[1], 1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
serie_size =  X_tr.shape[1]
model = Sequential()
model.add(LSTM(units = 64,input_shape = (serie_size,1)))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
model.summary()
history = model.fit(X_tr, y_tr, epochs=50, batch_size=72, validation_data=(X_val, y_val), verbose=2, shuffle=False)

In [ ]:
X_test = X_test.to_numpy().reshape(X_test.shape[0], X_test.shape[1], 1)
y_pred = model.predict(X_test).clip(0, 125000)

In [ ]:
sub['sales'] = np.exp(np.array(list(map(lambda x: max(x, 0), y_pred))))-1

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()